<a href="https://colab.research.google.com/github/aurioldegbelo/sis2024/blob/main/2024_SIS_Demo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Similarity Computation

In [1]:
def sum (a, b):
  return a+b


In [2]:
sum(1,3)

4